In [1]:
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import os

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

import re
import pandas as pd
from tabulate import tabulate

In [ ]:
url = 'https://codeup.com/codeups-data-science-career-accelerator-is-here/'
headers = {'User-Agent': 'Codeup Data Science'} # Some websites don't accept the python-requests default user-agent
response = get(url, headers=headers)

In [ ]:
response.text

In [ ]:
print(response.text[:400])

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
article = soup.find('div', class_='jupiterx-post-content')
article.text

In [ ]:
with open('article.txt', 'w') as f:
    f.write(article.text)

In [ ]:
def get_article_text():
    # if we already have the data, read it locally
    if os.path.exists('article.txt'):
        with open('article.txt') as f:
            return f.read()

    # otherwise go fetch the data
    url = 'https://codeup.com/codeups-data-science-career-accelerator-is-here/'
    headers = {'User-Agent': 'Codeup Data Science'}
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    title = soup.title.string
    article = soup.find('div', class_='jupiterx-post-content')

    # save it for next time
    with open('article.txt', 'w') as f:
        f.write(article.text)

    return title, article.text

In [ ]:
get_article_text()

In [ ]:
soup.find("h1")

In [ ]:
soup.find_all("a")

In [ ]:
soup.find_all('<a href="https')

In [ ]:
soup.a.text

In [ ]:
soup.a(2)

In [ ]:
soup.select('a[href]')

In [ ]:
def get_blog_articles()

In [ ]:
#launch url
url = "https://codeup.com/resources/#blog"

# create a new Safari session
browser = webdriver.Safari()
# makes the browser wait if it can't find an element
browser.implicitly_wait(10)
browser.get(url)

# from selenium.webdriver.common.action_chains import ActionChains

# link = browser.find_element_by_link_text('Read More')
# ActionChains(browser).move_to_element(link).perform()
# link.click()

In [ ]:
#After opening the url above, Selenium clicks the specific agency link
python_button = driver.find_element_by_id('#blog > div > div > div > div > div > div.elementor-element.elementor-element-d61b951.codeupblog.elementor-widget.elementor-widget-jet-listing-grid.is-mac > div > div > div > div.jet-listing-grid__item.jet-listing-dynamic-post-6110 > div > div > div > section > div > div > div > div > div > div.elementor-element.elementor-element-61bf69f.elementor-widget.elementor-widget-jet-listing-dynamic-link.is-mac > div > div > a') #FHSU
python_button.click() #click fhsu link

# #Selenium hands the page source to Beautiful Soup
# headers = {'User-Agent': 'Codeup Data Science'}
# soup_level1=BeautifulSoup(browser.page_source, 'lxml')

In [ ]:
datalist = [] #empty list
x = 0 #counter

#Beautiful Soup finds all Job Title links on the agency page and the loop begins
for link in soup_level1.find_all('a', id=re.compile("^MainContent_uxLevel2_JobTitles_uxJobTitleBtn_")):
    
    #Selenium visits each Job Title page
    python_button = driver.find_element_by_id('MainContent_uxLevel2_JobTitles_uxJobTitleBtn_' + str(x))
    python_button.click() #click link
    
    #Selenium hands of the source of the specific job page to Beautiful Soup
    soup_level2=BeautifulSoup(driver.page_source, 'lxml')

    #Beautiful Soup grabs the HTML table on the page
    table = soup_level2.find_all('table')[0]
    
    #Giving the HTML table to pandas to put in a dataframe object
    df = pd.read_html(str(table),header=0)
    
    #Store the dataframe in a list
    datalist.append(df[0])
    
    #Ask Selenium to click the back button
    driver.execute_script("window.history.go(-1)") 
    
    #increment the counter variable before starting the loop over
    x += 1
    
    #end loop block
    
#loop has completed

#end the Selenium browser session
driver.quit()

#combine all pandas dataframes in the list into one big dataframe
result = pd.concat([pd.DataFrame(datalist[i]) for i in range(len(datalist))],ignore_index=True)

#convert the pandas dataframe to JSON
json_records = result.to_json(orient='records')

#pretty print to CLI with tabulate
#converts to an ascii table
print(tabulate(result, headers=["Employee Name","Job Title","Overtime Pay","Total Gross Pay"],tablefmt='psql'))

#get current working directory
path = os.getcwd()

#open, write, and close the file
f = open(path + "\\fhsu_payroll_data.json","w") #FHSU
f.write(json_records)
f.close()

In [ ]:
def get_all_blogs():
    url = 'https://codeup.com/resources/#blog'
    headers = {'User-Agent': "Curie Data Science Cohort"}
    response = get(url,headers=headers)
    soup = BeautifulSoup(response.text)
    blogs = soup.find('div',class_="jet-listing jet-listing-dynamic-link")
    links = blogs.find_all(href=True)

    all_links = [link.attrs['href'] for link in links if re.findall(r'\d+\/\d+',link.attrs['href'])== [] and len(link.attrs['href'])>1]
    all_links = list(set(all_links))
    #loop to pull all codeup blogs
    #urls = ['https://codeup.com/codeups-data-science-career-accelerator-is-here/','https://codeup.com/data-science-myths/','https://codeup.com/data-science-vs-data-analytics-whats-the-difference/','https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/','https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

    df = pd.DataFrame()
    blogs = []
    titles = []
    for link in all_links:

        response = get(link,headers=headers)
        soup = BeautifulSoup(response.text)
        body = soup.find('div',class_="mk-single-content").get_text()
        if soup.find('h1',class_="page-title") is not None:
            title = soup.find('h1',class_="page-title").get_text()
        else:
            title = np.nan

        blogs.append(body)
        titles.append(title)
    df['title'] = pd.Series(titles)
    df['article'] = pd.Series(blogs)
    return df

In [ ]:
df = get_all_blogs()

In [ ]:
url = 'https://codeup.com/resources/#blog'
headers = {'User-Agent': "Curie Data Science Cohort"}
response = get(url,headers=headers)

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
blogs = soup.find('div',class_="jet-listing jet-listing-dynamic-link")

In [ ]:
links = blogs.find_all(href=True)

In [ ]:
links

In [ ]:
all_links = [link.attrs['href'] for link in links if re.findall(r'\d+\/\d+',link.attrs['href'])== [] and len(link.attrs['href'])>1]
all_links = list(set(all_links))

In [ ]:
def get_all_blogs():
    url = 'https://codeup.com/resources/#blog'
    headers = {'User-Agent': "Curie Data Science Cohort"}
    response = get(url,headers=headers)
    soup = BeautifulSoup(response.text)
    blogs = soup.find('div',class_="jet-listing jet-listing-dynamic-link")
    links = blogs.find_all(href=True)

    all_links = [link.attrs['href'] for link in links if re.findall(r'\d+\/\d+',link.attrs['href'])== [] and len(link.attrs['href'])>1]
    all_links = list(set(all_links))
    #loop to pull all codeup blogs
    #urls = ['https://codeup.com/codeups-data-science-career-accelerator-is-here/','https://codeup.com/data-science-myths/','https://codeup.com/data-science-vs-data-analytics-whats-the-difference/','https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/','https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

    df = pd.DataFrame()
    blogs = []
    titles = []
    for link in all_links:

        response = get(link,headers=headers)
        soup = BeautifulSoup(response.text)
        body = soup.find('div',class_="jupiterx-post-content").get_text()
        title = soup.find('h1',class_="jupiterx-post-title").get_text()
        

        blogs.append(body)
        titles.append(title)
    df['title'] = pd.Series(titles)
    df['article'] = pd.Series(blogs)
    return df

In [ ]:
df = get_all_blogs()

In [ ]:
df

In [42]:
url = "https://codeup.com/resources/#blog"

# create a new Safari session
browser = webdriver.Safari()
# makes the browser wait if it can't find an element
browser.implicitly_wait(10)
browser.get(url)

# #Selenium hands the page source to Beautiful Soup
# headers = {'User-Agent': 'Codeup Data Science'}
# soup_level=BeautifulSoup(browser.page_source, 'lxml')

url = 'https://codeup.com/resources/#blog'
headers = {'User-Agent': "Curie Data Science Cohort"}
response = get(url,headers=headers)
soup = BeautifulSoup(response.text)
blogs = soup.find('div',class_="jet-listing jet-listing-dynamic-link")

datalist = [] #empty list
x = 0 #counter

for link in blogs.find_all(href=True):
    
    all_links = [link.attrs['href'] for link in links if re.findall(r'\d+\/\d+',link.attrs['href'])== [] and len(link.attrs['href'])>1]
    all_links = list(set(all_links))
    
    #Selenium visits each Job Title page
    python_button = driver.find_element_by_id('MainContent_uxLevel2_JobTitles_uxJobTitleBtn_' + str(x))
    python_button.click() #click link
    
    #Selenium hands of the source of the specific job page to Beautiful Soup
    soup_level2=BeautifulSoup(driver.page_source, 'lxml')

    #Beautiful Soup grabs the HTML table on the page
    table = soup_level2.find_all('table')[0]
    
    #Giving the HTML table to pandas to put in a dataframe object
    df = pd.read_html(str(table),header=0)
    
    #Store the dataframe in a list
    datalist.append(df[0])
    
    #Ask Selenium to click the back button
    driver.execute_script("window.history.go(-1)") 
    
    #increment the counter variable before starting the loop over
    x += 1
    
def get_all_blogs():
    url = 'https://codeup.com/resources/#blog'
    headers = {'User-Agent': "Curie Data Science Cohort"}
    response = get(url,headers=headers)
    soup = BeautifulSoup(response.text)
    all_links = []
    
    for a in soup.find_all('a', class_="jet-listing jet-listing-dynamic-link", href=True):
        links.append(a['href'])
    #loop to pull all codeup blogs
    #urls = ['https://codeup.com/codeups-data-science-career-accelerator-is-here/','https://codeup.com/data-science-myths/','https://codeup.com/data-science-vs-data-analytics-whats-the-difference/','https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/','https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

    df = pd.DataFrame()
    blogs = []
    titles = []
    for link in all_links:

        response = get(link,headers=headers)
        soup = BeautifulSoup(response.text)
        body = soup.find('div',class_="jupiterx-post-content").get_text()
        title = soup.find('h1',class_="jupiterx-post-title").get_text()
        

        blogs.append(body)
        titles.append(title)
    df['title'] = pd.Series(titles)
    df['article'] = pd.Series(blogs)
    return df

WebDriverException: Message: 


In [ ]:
url = 'https://codeup.com/resources/#blog'
headers = {'User-Agent': "Curie Data Science Cohort"}
response = get(url,headers=headers)
soup = BeautifulSoup(response.text)
blogs = soup.find_all('div', class_="jet-listing jet-listing-dynamic-link")
links = blogs.find_all(href=True)

In [ ]:
blogs

In [ ]:
def get_articles_from_topic(url):
    headers = {'user-agent': 'Codeup Curie'}
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    output = []

    articles = soup.select(".news-card")

    for article in articles: 
        title = article.select("[itemprop='headline']")[0].get_text()
        body = article.select("[itemprop='articleBody']")[0].get_text()
        author = article.select(".author")[0].get_text()
        published_date = article.select(".time")[0]["content"]
        category = response.url.split("/")[-1]

        article_data = {
            'title': title,
            'body': body,
            'category': category,
            'author': author,
            'published_date': published_date,
        }
        output.append(article_data)


    return output


def make_new_request():
    urls = [
        "https://inshorts.com/en/read/business",
        "https://inshorts.com/en/read/sports",
        "https://inshorts.com/en/read/technology",
        "https://inshorts.com/en/read/entertainment"
    ]

    output = []
    
    for url in urls:
        # We use .extend in order to make a flat output list.
        output.extend(get_articles_from_topic(url))

    df = pd.DataFrame(output)
    df.to_csv('inshorts_news_articles.csv') 

    return df

In [ ]:
news = make_new_request()

In [ ]:
news.head()

In [31]:
url = 'https://codeup.com/resources/#blog'
headers = {'User-Agent': "Curie Data Science Cohort"}
response = get(url,headers=headers)
soup = BeautifulSoup(response.text)
blogs = soup.find_all('div',class_="jet-listing jet-listing-dynamic-link")
#links = blogs.find_all(href=True)

all_links = [link.attrs['href'] for link in blogs if re.findall(r'\d+\/\d+',link.attrs['href'])== [] and len(link.attrs['href'])>1]
all_links = list(set(all_links))

AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [30]:
all_links

['https://codeup.com/bootcamp-to-bootcamp/']

In [35]:
blogs

[<div class="jet-listing jet-listing-dynamic-link"><a class="jet-listing-dynamic-link__link" href="https://codeup.com/bootcamp-to-bootcamp/"><span class="jet-listing-dynamic-link__label">Read More</span></a></div>,
 <div class="jet-listing jet-listing-dynamic-link"><a class="jet-listing-dynamic-link__link" href="https://codeup.com/how-to-get-started-on-a-programming-exercise/"><span class="jet-listing-dynamic-link__label">Read More</span></a></div>,
 <div class="jet-listing jet-listing-dynamic-link"><a class="jet-listing-dynamic-link__link" href="https://codeup.com/career-in-data-science/"><span class="jet-listing-dynamic-link__label">Read More</span></a></div>,
 <div class="jet-listing jet-listing-dynamic-link"><a class="jet-listing-dynamic-link__link" href="https://codeup.com/getting-hired-in-a-remote-environment/"><span class="jet-listing-dynamic-link__label">Read More</span></a></div>,
 <div class="jet-listing jet-listing-dynamic-link"><a class="jet-listing-dynamic-link__link" href

In [29]:
links

[<a class="jet-listing-dynamic-link__link" href="https://codeup.com/bootcamp-to-bootcamp/"><span class="jet-listing-dynamic-link__label">Read More</span></a>]

In [ ]:
    #loop to pull all codeup blogs
    #urls = ['https://codeup.com/codeups-data-science-career-accelerator-is-here/','https://codeup.com/data-science-myths/','https://codeup.com/data-science-vs-data-analytics-whats-the-difference/','https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/','https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

    df = pd.DataFrame()
    blogs = []
    titles = []
    for link in all_links:

        response = get(link,headers=headers)
        soup = BeautifulSoup(response.text)
        body = soup.find('div',class_="mk-single-content").get_text()
        if soup.find('h1',class_="page-title") is not None:
            title = soup.find('h1',class_="page-title").get_text()
        else:
            title = np.nan

        blogs.append(body)
        titles.append(title)
    df['title'] = pd.Series(titles)
    df['article'] = pd.Series(blogs)
    return df

In [71]:
def get_all_blog_links():
    url = 'https://codeup.com/resources/#blog'
    headers = {'User-Agent': "Curie Data Science Cohort"}
    response = get(url,headers=headers)
    soup = BeautifulSoup(response.text)
    links = []
    
    for a in soup.find_all('a', class_="jet-listing jet-listing-dynamic-link", href=True):
        links.append(a['href'])
    return links

In [72]:
get_all_blog_links()

[]

In [58]:
def get_all_blog_links():
    url = 'https://codeup.com/resources/#blog'
    headers = {'User-Agent': "Curie Data Science Cohort"}
    response = get(url,headers=headers)
    soup = BeautifulSoup(response.text)
    
    links = []
    
    for a in soup.find_all('a', class_="jet-listing jet-listing-dynamic-link", href=True):
        links.append(a['href'])
        
        return links
    #loop to pull all codeup blogs
    #urls = ['https://codeup.com/codeups-data-science-career-accelerator-is-here/','https://codeup.com/data-science-myths/','https://codeup.com/data-science-vs-data-analytics-whats-the-difference/','https://codeup.com/10-tips-to-crush-it-at-the-sa-tech-job-fair/','https://codeup.com/competitor-bootcamps-are-closing-is-the-model-in-danger/']

    df = pd.DataFrame()
    blogs = []
    titles = []
    for link in all_links:
        headers = {'User-Agent': "Curie Data Science Cohort"}
        response = get(link,headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        body = soup.find('div',class_="jupiterx-post-content").get_text()
        title = soup.find('h1',class_="jupiterx-post-title").get_text()
        

        blogs.append(body)
        titles.append(title)
    df['title'] = pd.Series(titles)
    df['article'] = pd.Series(blogs)
    return df

In [59]:
df = get_all_blogs()

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


In [53]:
get_all_blog_links()

[]

In [68]:
#def get_all_blog_links():
url = 'https://codeup.com/resources/#blog'
headers = {'User-Agent': "Curie Data Science Cohort"}
response = get(url,headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

links = []

for a in soup.find_all('a', class_="jet-listing jet-listing-dynamic-link", href=True):
    return links.append(a['href'])

SyntaxError: 'return' outside function (<ipython-input-68-0ec8248f2d43>, line 10)

In [61]:
get_all_blog_links()

[]

In [70]:
links

[]

In [76]:
def get_news_articles():
    filename = 'inshorts_news_articles.csv'

    # check for presence of the file or make a new request
    if os.path.exists(filename):
        return pd.read_csv(filename)
    else:
        return make_new_request()

def get_articles_from_topic(url):
    headers = {'user-agent': 'Curie Data Science'}
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')

    output = []

    articles = soup.select(".news-card")

    for article in articles: 
        title = article.select("[itemprop='headline']")[0].get_text()
        body = article.select("[itemprop='articleBody']")[0].get_text()
        author = article.select(".author")[0].get_text()
        published_date = article.select(".time")[0]["content"]
        category = response.url.split("/")[-1]

        article_data = {
            'title': title,
            'body': body,
            'category': category,
            'author': author,
            'published_date': published_date,
        }
        output.append(article_data)


    return output


def make_new_request():
    urls = [
        "https://inshorts.com/en/read/business",
        "https://inshorts.com/en/read/sports",
        "https://inshorts.com/en/read/technology",
        "https://inshorts.com/en/read/entertainment"
    ]

    output = []
    
    for url in urls:
        # We use .extend in order to make a flat output list.
        output.extend(get_articles_from_topic(url))

    df = pd.DataFrame(output)
    df.to_csv('inshorts_news_articles.csv') 

    return df

In [77]:
df = make_new_request()

In [79]:
df

,title,body,category,author,published_date
0,"Twitter CEO donates $10M to project giving $1,...",Twitter's billionaire CEO Jack Dorsey has dona...,business,Pragya Swastik,2020-05-27T09:16:22.000Z
1,Nokia shuts plant in Tamil Nadu after some emp...,Nokia on Tuesday said it suspended operations ...,business,Krishna Veera Vanamali,2020-05-27T04:10:52.000Z
2,Firm whose stock surged 1000% in 2020 starts h...,US biotech company Novavax said it has started...,business,Krishna Veera Vanamali,2020-05-26T10:17:41.000Z
3,Glenmark to start clinical trial to test poten...,Glenmark Pharmaceuticals on Tuesday said it wi...,business,Krishna Veera Vanamali,2020-05-26T11:22:33.000Z
4,India's economic growth seen at 1.2% in Q4 FY2...,India's economy is estimated to have grown at ...,business,Dharna,2020-05-26T12:41:03.000Z
...,...,...,...,...,...
95,What's happening is a grim scenario: Ranveer o...,Actor Ranveer Singh has said that he has been ...,entertainment,Atul Mishra,2020-05-27T04:27:19.000Z
96,"Salman Khan sends sweets' ingredients to 5,000...","On the occasion of Eid, Salman Khan sent ingre...",entertainment,Atul Mishra,2020-05-27T06:26:20.000Z
97,Actor Kiran Kumar tests -ve for COVID-19 after...,"Actor Kiran Kumar, who tested positive for cor...",entertainment,Udit Gupta,2020-05-27T10:52:02.000Z
98,Suicide Squad director's cut release would be ...,"Director David Ayer has that it would be ""incr...",entertainment,Atul Mishra,2020-05-27T11:10:41.000Z


In [81]:
df.body

0     Twitter's billionaire CEO Jack Dorsey has dona...
1     Nokia on Tuesday said it suspended operations ...
2     US biotech company Novavax said it has started...
3     Glenmark Pharmaceuticals on Tuesday said it wi...
4     India's economy is estimated to have grown at ...
                            ...                        
95    Actor Ranveer Singh has said that he has been ...
96    On the occasion of Eid, Salman Khan sent ingre...
97    Actor Kiran Kumar, who tested positive for cor...
98    Director David Ayer has that it would be "incr...
99    Actor Himansh Kohli has revealed that he flew ...
Name: body, Length: 100, dtype: object

In [82]:
url = 'https://codeup.com/resources/#blog'
headers = {'User-Agent': 'Codeup Data Science'}
response = get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

links = []

for a in soup.find_all('a', class_='jet-listing-dynamic-link__link', href=True):
    links.append(a['href'])

In [ ]:
articles = []

for link in links:
    headers = {'User-Agent': 'Codeup Data Science'}
    response = get(link, headers=headers)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    article_text = soup.find('div', class_='jupiterx-post-content clearfix').text
    article_title = soup.find('h1', class_='jupiterx-post-title').text

    article_dict = {'title': article_title,
                    'content': article_text}
    
    articles.append(article_dict)

articles